## 1. Setup e Imports

In [1]:
import sys
import torch
import torch.nn as nn
from torchvision import models, transforms
from transformers import RobertaTokenizer, RobertaModel
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

# Fuzzy system
sys.path.insert(0, '/home/paloma/cerebrum-artis')
from cerebrum_artis.fuzzy.fuzzy_brain.extractors.visual import VisualFeatureExtractor

# Style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Imports concluídos!")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")

/data/paloma/venvs/cerebrum-artis/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


✅ Imports concluídos!
🖥️  Device: cuda


## 2. Definir Modelo V2

In [2]:
EMOTIONS = [
    'amusement', 'awe', 'contentment', 'excitement',
    'anger', 'disgust', 'fear', 'sadness', 'something else'
]

class V2_SimpleConcat(nn.Module):
    """V2: ResNet50 + RoBERTa + 7 Fuzzy Features"""
    
    def __init__(self, num_classes=9, dropout=0.3):
        super().__init__()
        
        # Vision: ResNet50 (FROZEN)
        resnet = models.resnet50(weights='IMAGENET1K_V1')
        self.visual_encoder = nn.Sequential(*list(resnet.children())[:-1])
        for param in self.visual_encoder.parameters():
            param.requires_grad = False
        
        # Text: RoBERTa (FROZEN)
        self.text_encoder = RobertaModel.from_pretrained('roberta-base')
        for param in self.text_encoder.parameters():
            param.requires_grad = False
        
        # Fusion MLP (TRAINABLE)
        # 2048 (ResNet) + 768 (RoBERTa) + 7 (Fuzzy) = 2823
        self.fusion = nn.Sequential(
            nn.Linear(2048 + 768 + 7, 1024),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, image, input_ids, attention_mask, fuzzy_features):
        # Visual: [B, 2048]
        visual_feats = self.visual_encoder(image).view(image.size(0), -1)
        
        # Text: [B, 768]
        text_output = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        text_feats = text_output.last_hidden_state[:, 0, :]  # [CLS] token
        
        # Concatenar: [B, 2823]
        combined = torch.cat([visual_feats, text_feats, fuzzy_features], dim=1)
        
        # Classificar
        logits = self.fusion(combined)
        return logits

print("✅ Modelo V2 definido!")

✅ Modelo V2 definido!


## 3. Carregar Modelo Treinado

In [3]:
# Criar modelo
model = V2_SimpleConcat(num_classes=9, dropout=0.3).to(device)

# Carregar checkpoint (ajuste o path conforme necessário)
checkpoint_path = '/data/paloma/deep-mind-checkpoints/v2_fuzzy_features/best_model.pt'

if Path(checkpoint_path).exists():
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✅ Checkpoint carregado: {checkpoint_path}")
    print(f"📊 Epoch: {checkpoint.get('epoch', 'N/A')}")
    print(f"📊 Val Accuracy: {checkpoint.get('val_acc', 'N/A'):.2f}%")
else:
    print(f"⚠️  Checkpoint não encontrado: {checkpoint_path}")
    print("Continuando com pesos aleatórios (apenas para demonstração)")

model.eval()

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n📊 Total params: {total_params:,}")
print(f"📊 Trainable params: {trainable_params:,}")
print(f"📊 Frozen params: {total_params - trainable_params:,}")

/data/paloma/venvs/cerebrum-artis/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of R

⚠️  Checkpoint não encontrado: /data/paloma/deep-mind-checkpoints/v2_fuzzy_features/best_model.pt
Continuando com pesos aleatórios (apenas para demonstração)

📊 Total params: 151,574,857
📊 Trainable params: 3,421,193
📊 Frozen params: 148,153,664


## 4. Setup de Preprocessamento

In [4]:
# Image transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Fuzzy extractor
fuzzy_extractor = VisualFeatureExtractor()

print("✅ Preprocessamento configurado!")

✅ Preprocessamento configurado!


## 5. Função de Predição

In [5]:
def predict_emotion(image_path, utterance):
    """
    Prediz emoção para uma imagem + utterance.
    
    Args:
        image_path: Path para imagem
        utterance: Texto descritivo
    
    Returns:
        dict com predição, probabilidades, features fuzzy
    """
    # Load image
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)  # [1, 3, 224, 224]
    
    # Extract fuzzy features
    fuzzy_dict = fuzzy_extractor.extract(str(image_path))
    fuzzy_features = torch.tensor([
        fuzzy_dict['brightness'],
        fuzzy_dict['color_temperature'],
        fuzzy_dict['saturation'],
        fuzzy_dict['color_harmony'],
        fuzzy_dict['complexity'],
        fuzzy_dict['symmetry'],
        fuzzy_dict['texture_roughness']
    ], dtype=torch.float32).unsqueeze(0).to(device)  # [1, 7]
    
    # Tokenize text
    tokens = tokenizer(
        utterance,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)
    
    # Forward pass
    with torch.no_grad():
        logits = model(image_tensor, input_ids, attention_mask, fuzzy_features)
        probs = torch.softmax(logits, dim=1)
    
    # Results
    pred_idx = torch.argmax(probs, dim=1).item()
    pred_emotion = EMOTIONS[pred_idx]
    confidence = probs[0, pred_idx].item()
    
    return {
        'predicted_emotion': pred_emotion,
        'confidence': confidence,
        'probabilities': {EMOTIONS[i]: probs[0, i].item() for i in range(9)},
        'fuzzy_features': fuzzy_dict,
        'image': image
    }

print("✅ Função de predição criada!")

✅ Função de predição criada!


## 6. Teste com Imagens de Exemplo

### 6.1 Starry Night (Van Gogh)

In [6]:
# Exemplo 1: Starry Night
image_path = '/data/paloma/data/paintings/wikiart/vincent-van-gogh_the-starry-night-1889.jpg'
utterance = "The swirling sky with vibrant blues and yellows evokes a sense of awe and wonder"

if Path(image_path).exists():
    result = predict_emotion(image_path, utterance)
    
    # Visualizar
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Imagem
    axes[0].imshow(result['image'])
    axes[0].axis('off')
    axes[0].set_title('Starry Night - Van Gogh', fontsize=14, fontweight='bold')
    
    # Probabilidades
    emotions = list(result['probabilities'].keys())
    probs = list(result['probabilities'].values())
    colors = ['#2ecc71' if e == result['predicted_emotion'] else '#95a5a6' for e in emotions]
    
    axes[1].barh(emotions, probs, color=colors)
    axes[1].set_xlabel('Probability', fontsize=12)
    axes[1].set_title(f'Prediction: {result["predicted_emotion"].upper()} ({result["confidence"]:.1%})', 
                      fontsize=14, fontweight='bold')
    axes[1].set_xlim(0, 1)
    axes[1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('/home/paloma/cerebrum-artis/outputs/figures/v2_starry_night.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print fuzzy features
    print("\n📊 FUZZY FEATURES:")
    for feat, val in result['fuzzy_features'].items():
        print(f"  {feat:20s}: {val:.3f}")
else:
    print(f"⚠️  Imagem não encontrada: {image_path}")

⚠️  Imagem não encontrada: /data/paloma/data/paintings/wikiart/vincent-van-gogh_the-starry-night-1889.jpg


### 6.2 Visualizar Features Fuzzy

In [7]:
if Path(image_path).exists():
    # Plot fuzzy features
    features = result['fuzzy_features']
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    feature_names = list(features.keys())
    feature_values = list(features.values())
    
    bars = ax.barh(feature_names, feature_values, color='#3498db')
    
    # Highlight high values
    for i, (name, val) in enumerate(zip(feature_names, feature_values)):
        if val > 0.7:
            bars[i].set_color('#e74c3c')  # Red for high
        elif val < 0.3:
            bars[i].set_color('#95a5a6')  # Gray for low
    
    ax.set_xlabel('Value [0, 1]', fontsize=12)
    ax.set_title('Fuzzy Features - Starry Night', fontsize=14, fontweight='bold')
    ax.set_xlim(0, 1)
    ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('/home/paloma/cerebrum-artis/outputs/figures/v2_fuzzy_features_starry.png', 
                dpi=300, bbox_inches='tight')
    plt.show()

## 7. Comparar Múltiplas Pinturas

In [8]:
# Lista de exemplos (ajuste conforme suas imagens disponíveis)
examples = [
    {
        'path': '/data/paloma/data/paintings/wikiart/vincent-van-gogh_the-starry-night-1889.jpg',
        'utterance': 'The swirling sky evokes awe and wonder',
        'name': 'Starry Night'
    },
    # Adicione mais exemplos aqui
]

results_list = []

for ex in examples:
    if Path(ex['path']).exists():
        result = predict_emotion(ex['path'], ex['utterance'])
        result['name'] = ex['name']
        results_list.append(result)
        print(f"✅ {ex['name']}: {result['predicted_emotion']} ({result['confidence']:.1%})")

print(f"\n📊 Total processado: {len(results_list)} imagens")


📊 Total processado: 0 imagens


## 8. Análise de Contribuição de Features

Visualizar como diferentes features contribuem para a predição.

In [9]:
if results_list:
    # Create feature matrix
    feature_names = ['brightness', 'color_temp', 'saturation', 'harmony', 
                     'complexity', 'symmetry', 'roughness']
    
    feature_matrix = []
    painting_names = []
    
    for res in results_list:
        features = res['fuzzy_features']
        feature_matrix.append([features[f] for f in feature_names])
        painting_names.append(res['name'])
    
    # Heatmap
    fig, ax = plt.subplots(figsize=(10, len(painting_names)*0.8))
    
    sns.heatmap(
        feature_matrix,
        annot=True,
        fmt='.2f',
        cmap='RdYlBu_r',
        xticklabels=feature_names,
        yticklabels=painting_names,
        cbar_kws={'label': 'Feature Value'},
        vmin=0,
        vmax=1,
        ax=ax
    )
    
    ax.set_title('Fuzzy Features Comparison', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('/home/paloma/cerebrum-artis/outputs/figures/v2_features_heatmap.png', 
                dpi=300, bbox_inches='tight')
    plt.show()

## 9. Arquitetura do Modelo (Diagrama)

Visualizar a arquitetura V2.

In [10]:
# Criar diagrama de arquitetura textual
architecture_text = """
┌─────────────────────────────────────────────────────────────┐
│                    V2 - SIMPLE CONCATENATION                │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  INPUT: Image + Utterance                                   │
│                                                             │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐     │
│  │   ResNet50   │  │   RoBERTa    │  │    Fuzzy     │     │
│  │   (FROZEN)   │  │   (FROZEN)   │  │  Extractor   │     │
│  │   23M params │  │  125M params │  │  (7 features)│     │
│  └──────┬───────┘  └──────┬───────┘  └──────┬───────┘     │
│         │                 │                 │              │
│         ↓                 ↓                 ↓              │
│    [B, 2048]         [B, 768]           [B, 7]            │
│         │                 │                 │              │
│         └─────────────────┴─────────────────┘              │
│                           │                                │
│                    [B, 2823]                               │
│                           │                                │
│                           ↓                                │
│              ┌────────────────────────┐                    │
│              │    MLP Fusion          │                    │
│              │    (TRAINABLE)         │                    │
│              │    ~3M params          │                    │
│              │                        │                    │
│              │  2823 → 1024 → 512 → 9│                    │
│              └────────────┬───────────┘                    │
│                           │                                │
│                           ↓                                │
│                    [B, 9 emotions]                         │
│                                                             │
│  PERFORMANCE: 70.63% accuracy                              │
└─────────────────────────────────────────────────────────────┘
"""

print(architecture_text)

# Save as text file
with open('/home/paloma/cerebrum-artis/outputs/v2_architecture.txt', 'w') as f:
    f.write(architecture_text)

print("\n✅ Arquitetura salva em outputs/v2_architecture.txt")


┌─────────────────────────────────────────────────────────────┐
│                    V2 - SIMPLE CONCATENATION                │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  INPUT: Image + Utterance                                   │
│                                                             │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐     │
│  │   ResNet50   │  │   RoBERTa    │  │    Fuzzy     │     │
│  │   (FROZEN)   │  │   (FROZEN)   │  │  Extractor   │     │
│  │   23M params │  │  125M params │  │  (7 features)│     │
│  └──────┬───────┘  └──────┬───────┘  └──────┬───────┘     │
│         │                 │                 │              │
│         ↓                 ↓                 ↓              │
│    [B, 2048]         [B, 768]           [B, 7]            │
│         │                 │                 │              │
│         └─────────────────┴─────────────────┘       

## 10. Performance Summary

In [11]:
# Create performance summary
summary = {
    'Model': 'V2 - Simple Concatenation',
    'Architecture': 'ResNet50 (frozen) + RoBERTa (frozen) + 7 Fuzzy',
    'Total Parameters': f'{total_params:,}',
    'Trainable Parameters': f'{trainable_params:,}',
    'Frozen Parameters': f'{total_params - trainable_params:,}',
    'Val Accuracy': '70.63%',
    'Training Strategy': 'Transfer Learning (frozen encoders)',
    'Fusion Method': 'Simple Concatenation',
    'Key Features': '• Interpretable fuzzy features\n• Fast training (~hours)\n• Low computational cost'
}

print("\n" + "="*60)
print("V2 MODEL SUMMARY")
print("="*60)
for key, value in summary.items():
    print(f"{key:25s}: {value}")
print("="*60)


V2 MODEL SUMMARY
Model                    : V2 - Simple Concatenation
Architecture             : ResNet50 (frozen) + RoBERTa (frozen) + 7 Fuzzy
Total Parameters         : 151,574,857
Trainable Parameters     : 3,421,193
Frozen Parameters        : 148,153,664
Val Accuracy             : 70.63%
Training Strategy        : Transfer Learning (frozen encoders)
Fusion Method            : Simple Concatenation
Key Features             : • Interpretable fuzzy features
• Fast training (~hours)
• Low computational cost


## 11. Export para Apresentação

Todas as figuras foram salvas em:
- `outputs/figures/v2_starry_night.png`
- `outputs/figures/v2_fuzzy_features_starry.png`
- `outputs/figures/v2_features_heatmap.png`
- `outputs/v2_architecture.txt`

Use essas imagens na sua apresentação! 🎨